In [2]:
# common library
import os
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import argparse
import json
import cv2

# torch library
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models

# import my file
from lib.data_utils import test

1-10是simulator的数据集，11-14是Intraoperative的数据集

In [34]:

# base path 
base_path = 'Traindata/'
# simulator data set
simulator_num = [i+1 for i in range(10)]
s_tmp = '1'
simulator_path = base_path + 'aicm'+s_tmp+'/VID000_0'
# intraoperative data set
intra_num = [11, 12, 13, 14]
i_tmp = '11'
intra_path = 'aicm' + i_tmp

images_path = simulator_path + '/images'
labels_path = simulator_path + '/point_labels'

print(images_path)
print(labels_path)

Traindata/aicm1/VID000_0/images
Traindata/aicm1/VID000_0/point_labels


In [46]:
images_name = os.listdir(images_path)
labels_name = os.listdir(labels_path)
file_name = []
for i in images_name:
    file_name.append(i.split(".")[0])
#     print(i.split(".")[0])
# print(file_name)
# print(images_name)
# print(labels_name)




In [11]:
test()

I am in utils


In [11]:
import json
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def label2mask(file_name):
    file_in = json.load(open(file_name))
    points = file_in["points"]
    point_outs = []
    height = file_in["imageHeight"]
    width = file_in["imageWidth"]
    img = np.zeros((height,width,3),dtype = np.uint8)
    for point in points:
        point_out = []
        point_out.append(point["x"])
        point_out.append(point["y"])
        point_outs.append(point_out)
    for a in point_outs:
        img[a[0],a[1]] = [255,255,255]
        for i in range(6):
            img[a[0]+i,a[1]+i] = [255,255,255]
            img[a[0]+i,a[1]-i] = [255,255,255]
            img[a[0]-i,a[1]+i] = [255,255,255]
            img[a[0]-i,a[1]-i] = [255,255,255]
    img_show = Image.fromarray(img,"RGB")
    img_show.show()
    img_show.save("./cyclemap.png")


In [12]:
label2mask("./Traindata/aicm2/VID000_0/point_labels/sim_000000.json")

In [35]:
import matplotlib.pyplot as plt
import numpy as np
import cv2



# target = np.zeros((288, 512))
# plt.imshow(target, cmap='hot', interpolation='nearest')
# plt.show()
# target = generate_heatmap(target, (15, 15))
# plt.imshow(target, cmap='hot', interpolation='nearest')
# plt.show()
# # print(target)

In [36]:
def generate_heatmap(img, x, y, sigma=(15,15)):
    img[x][y] = 1
    img = cv2.GaussianBlur(img, sigma, 2)
    am = np.amax(img)
    img /= am / 255
    heatmap = img
    return heatmap



def heatmap_generator(file_name):
    file_in = json.load(open(file_name))
    points = file_in["points"]
    point_outs = []
    height = file_in["imageHeight"]
    width = file_in["imageWidth"]
    img = np.zeros((height,width,3),dtype = np.uint8)
    for point in points:
        point_out = []
        point_out.append(point["x"])
        point_out.append(point["y"])
        point_outs.append(point_out)
    
    x, y = point_outs[0]
    img = generate_heatmap(img, x, y)

    img_show = Image.fromarray(img,"RGB")
    img_show.show()
    img_show.save("./pic/heatmap.png")



In [37]:
heatmap_generator("./Traindata/aicm2/VID000_0/point_labels/sim_000000.json")

TypeError: ufunc 'true_divide' output (typecode 'd') could not be coerced to provided output parameter (typecode 'B') according to the casting rule ''same_kind''